In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import warnings
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

warnings.filterwarnings('ignore')

# Import StatsBomb classes - safe to run multiple times
try:
    from src.utils.config import StatsBombConfig
    from src.utils.data_fetcher import StatsBombDataFetcher
    print("✓ Imports successful")
except ImportError as e:
    print(f"✗ Import error: {e}")
    print("Make sure you're running this from the notebooks directory")

# Initialize StatsBomb classes - safe to run multiple times
try:
    config = StatsBombConfig()
    fetcher = StatsBombDataFetcher()
    print("✓ StatsBomb classes initialized successfully")
except Exception as e:
    print(f"✗ Initialization error: {e}")
    print("Check your .env file for STATSBOMB_USERNAME and STATSBOMB_PASSWORD")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

✓ Imports successful
Usuario configurado: itam_hackathon@hudl.com
Usuario configurado: itam_hackathon@hudl.com
✓ StatsBomb classes initialized successfully


In [2]:
fetcher = StatsBombDataFetcher()
SEASONS = [
    (317, '2024/2025'),
    (281, '2023/2024'),
    (235, '2022/2023'),
    (108, '2021/2022')
]
COMPETITION_ID = 73  # Liga MX

Usuario configurado: itam_hackathon@hudl.com


In [3]:
def extract_player_key_metrics(player_season_df: pd.DataFrame) -> pd.DataFrame:
    """
    Extrae y calcula métricas clave de jugadores para el sistema de recomendación.
    
    Args:
        player_season_df: DataFrame con estadísticas de jugadores por temporada
        
    Returns:
        DataFrame con métricas procesadas y normalizadas
    """
    
    # Filtrar jugadores con minutos mínimos (al menos 450 minutos = ~5 partidos completos)
    df = player_season_df[player_season_df['player_season_minutes'] >= 450].copy()
    
    # Métricas básicas
    base_metrics = [
        'account_id', 'player_id', 'player_name', 'team_id', 'team_name',
        'season_id', 'season_name', 'country_id', 'player_season_minutes',
        'primary_position_id', 'primary_position'
    ]
    
    # Métricas ofensivas (por 90 minutos)
    offensive_metrics = [
        'player_season_goals_90',
        'player_season_assists_90',
        'player_season_np_xg_90',
        'player_season_xag_90',
        'player_season_np_shots_90',
        'player_season_np_xg_per_shot',
        'player_season_shot_touch_ratio',
        'player_season_dribbles_90',
        'player_season_dribble_ratio'
    ]
    
    # Métricas de pases y creación
    passing_metrics = [
        'player_season_passes_into_box_90',
        'player_season_cross_completion_ratio',
        'player_season_deep_completions_90',
        'player_season_key_passes_90',
        'player_season_pass_completion_ratio',
        'player_season_progressive_passes_90',
        'player_season_obv_pass_90'
    ]
    
    # Métricas defensivas
    defensive_metrics = [
        'player_season_pressures_90',
        'player_season_pressure_regains_90',
        'player_season_tackles_90',
        'player_season_interceptions_90',
        'player_season_blocks_90',
        'player_season_clearances_90',
        'player_season_defensive_actions_90',
        'player_season_aerial_ratio'
    ]
    
    # Métricas de portero
    gk_metrics = [
        'player_season_psxg_conceded',
        'player_season_save_ratio',
        'player_season_clean_sheet_ratio'
    ]
    
    # Métricas avanzadas (OBV, xG)
    advanced_metrics = [
        'player_season_obv_90',
        'player_season_obv_dribble_carry_90',
        'player_season_obv_defensive_action_90',
        'player_season_obv_shot_90'
    ]
    
    # Combinar todas las métricas disponibles
    all_metrics = base_metrics.copy()
    
    # Agregar solo las métricas que existen en el DataFrame
    for metric_list in [offensive_metrics, passing_metrics, defensive_metrics, 
                        gk_metrics, advanced_metrics]:
        for metric in metric_list:
            if metric in df.columns:
                all_metrics.append(metric)
    
    # Seleccionar columnas disponibles
    available_metrics = [col for col in all_metrics if col in df.columns]
    df_processed = df[available_metrics].copy()
    
    # Rellenar NaN con 0 para métricas numéricas
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
    df_processed[numeric_cols] = df_processed[numeric_cols].fillna(0)
    
    return df_processed

In [4]:
def classify_player_position(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clasifica jugadores en categorías de posición amplias.
    
    Args:
        df: DataFrame con datos de jugadores
        
    Returns:
        DataFrame con columna adicional 'position_category'
    """
    
    df = df.copy()
    
    # Mapeo de posiciones a categorías
    position_mapping = {
        # Porteros
        'Goalkeeper': 'GK',
        
        # Defensas
        'Right Back': 'DEF',
        'Left Back': 'DEF',
        'Right Center Back': 'DEF',
        'Left Center Back': 'DEF',
        'Center Back': 'DEF',
        'Right Wing Back': 'DEF',
        'Left Wing Back': 'DEF',
        
        # Mediocampistas
        'Right Defensive Midfield': 'MED',
        'Left Defensive Midfield': 'MED',
        'Center Defensive Midfield': 'MED',
        'Right Center Midfield': 'MED',
        'Left Center Midfield': 'MED',
        'Center Midfield': 'MED',
        'Right Midfield': 'MED',
        'Left Midfield': 'MED',
        'Right Attacking Midfield': 'MED',
        'Left Attacking Midfield': 'MED',
        'Center Attacking Midfield': 'MED',
        
        # Delanteros
        'Right Wing': 'FWD',
        'Left Wing': 'FWD',
        'Right Center Forward': 'FWD',
        'Left Center Forward': 'FWD',
        'Center Forward': 'FWD',
        'Secondary Striker': 'FWD'
    }
    
    if 'primary_position' in df.columns:
        df['position_category'] = df['primary_position'].map(position_mapping)
        df['position_category'] = df['position_category'].fillna('MED')  # Default a mediocampo
    
    return df

In [5]:
def normalize_metrics_by_position(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normaliza métricas usando z-score dentro de cada categoría de posición.
    Esto permite comparar jugadores entre diferentes posiciones.
    
    Args:
        df: DataFrame con métricas de jugadores
        
    Returns:
        DataFrame con métricas normalizadas (sufijo _norm)
    """
    
    df = df.copy()
    
    # Identificar columnas numéricas que terminan en _90
    metric_cols = [col for col in df.columns if col.endswith('_90') or col.endswith('_ratio')]
    
    # Normalizar por posición
    if 'position_category' in df.columns:
        for metric in metric_cols:
            if metric in df.columns:
                # Calcular z-score por posición
                df[f'{metric}_norm'] = df.groupby('position_category')[metric].transform(
                    lambda x: (x - x.mean()) / x.std() if x.std() > 0 else 0
                )
    
    return df

In [6]:
all_seasons_data = []

for season_id, season_name in SEASONS:
    print(f"\n{'=' * 60}")
    print(f"Procesando temporada: {season_name}")
    print('=' * 60)
    
    # Obtener datos de jugadores
    player_data = fetcher.get_player_season_stats(COMPETITION_ID, season_id)
    
    if player_data.empty:
        print(f"No hay datos para {season_name}")
        continue
    
    print(f"{len(player_data)} jugadores encontrados")
    
    # Extraer métricas clave
    processed_data = extract_player_key_metrics(player_data)
    print(f"Métricas extraídas: {len(processed_data)} jugadores con minutos suficientes")
    
    # Clasificar por posición
    processed_data = classify_player_position(processed_data)
    
    # Normalizar métricas
    processed_data = normalize_metrics_by_position(processed_data)
    
    print(f"Datos procesados y normalizados")
    print(f"   - Porteros: {len(processed_data[processed_data['position_category'] == 'GK'])}")
    print(f"   - Defensas: {len(processed_data[processed_data['position_category'] == 'DEF'])}")
    print(f"   - Mediocampistas: {len(processed_data[processed_data['position_category'] == 'MED'])}")
    print(f"   - Delanteros: {len(processed_data[processed_data['position_category'] == 'FWD'])}")
    
    all_seasons_data.append(processed_data)

# Combinar todas las temporadas
print(f"\n{'=' * 60}")
print("CONSOLIDANDO DATOS DE TODAS LAS TEMPORADAS")
print('=' * 60)

df_all_players = pd.concat(all_seasons_data, ignore_index=True)
print(f"Total de registros: {len(df_all_players)}")
print(f"Jugadores únicos: {df_all_players['player_id'].nunique()}")
print(f"Equipos únicos: {df_all_players['team_name'].nunique()}")


Procesando temporada: 2024/2025
Obteniendo estadísticas de los jugadores para (Comp: 73, Temp: 317)...
Estadísticas de 636 jugadores obtenidas
636 jugadores encontrados
Métricas extraídas: 425 jugadores con minutos suficientes
Datos procesados y normalizados
   - Porteros: 30
   - Defensas: 76
   - Mediocampistas: 268
   - Delanteros: 51

Procesando temporada: 2023/2024
Obteniendo estadísticas de los jugadores para (Comp: 73, Temp: 281)...
Estadísticas de 623 jugadores obtenidas
623 jugadores encontrados
Métricas extraídas: 419 jugadores con minutos suficientes
Datos procesados y normalizados
   - Porteros: 29
   - Defensas: 85
   - Mediocampistas: 268
   - Delanteros: 37

Procesando temporada: 2022/2023
Obteniendo estadísticas de los jugadores para (Comp: 73, Temp: 235)...
Estadísticas de 591 jugadores obtenidas
591 jugadores encontrados
Métricas extraídas: 416 jugadores con minutos suficientes
Datos procesados y normalizados
   - Porteros: 29
   - Defensas: 74
   - Mediocampistas: 2

In [7]:
america_players = df_all_players[df_all_players['team_name'] == 'América'].copy()

print(f"\n{'=' * 60}")
print("JUGADORES DEL CLUB AMÉRICA")
print('=' * 60)
print(f"Total de registros: {len(america_players)}")
print(f"Jugadores únicos: {america_players['player_id'].nunique()}")

# Jugadores por temporada
print("\nDistribución por temporada:")
print(america_players.groupby('season_name')['player_id'].count())

# Jugadores por posición
print("\nDistribución por posición:")
print(america_players.groupby('position_category')['player_id'].count())

# Top 10 jugadores del América por minutos (última temporada)
latest_season = america_players[america_players['season_name'] == '2024/2025']
if not latest_season.empty:
    print(f"\nTop 10 jugadores por minutos (2024/2025):")
    top_players = latest_season.nlargest(10, 'player_season_minutes')[
        ['player_name', 'primary_position', 'player_season_minutes', 'team_name']
    ]
    print(top_players.to_string(index=False))


JUGADORES DEL CLUB AMÉRICA
Total de registros: 93
Jugadores únicos: 45

Distribución por temporada:
season_name
2021/2022    22
2022/2023    23
2023/2024    25
2024/2025    23
Name: player_id, dtype: int64

Distribución por posición:
position_category
DEF    18
FWD    14
GK      8
MED    53
Name: player_id, dtype: int64

Top 10 jugadores por minutos (2024/2025):
                        player_name           primary_position  player_season_minutes team_name
       Luis Ángel Malagón Velázquez                 Goalkeeper              4236.8840   América
                Israel Reyes Romero          Right Centre Back              3757.0496   América
           Álvaro Fidalgo Fernández  Left Defensive Midfielder              3749.6165   América
        Alejandro Zendejas Saavedra                 Right Wing              3422.2002   América
          Ramón Juárez Del Castillo          Right Centre Back              3018.2832   América
      Erick Daniel Sánchez Ocegueda Right Defensive Midfie

In [8]:
def create_team_profile(df: pd.DataFrame, team_name: str, 
                       season_name: str = None) -> pd.DataFrame:
    """
    Crea un perfil promedio del equipo basado en sus jugadores.
    
    Args:
        df: DataFrame con datos de jugadores
        team_name: Nombre del equipo
        season_name: Temporada específica (opcional)
        
    Returns:
        DataFrame con perfil promedio por posición
    """
    
    # Filtrar por equipo
    team_df = df[df['team_name'] == team_name].copy()
    
    # Filtrar por temporada si se especifica
    if season_name:
        team_df = team_df[team_df['season_name'] == season_name]
    
    # Métricas a promediar (solo las normalizadas)
    metric_cols = [col for col in team_df.columns if col.endswith('_norm')]
    
    # Calcular promedios por posición
    profile = team_df.groupby('position_category')[metric_cols].mean()
    
    return profile

# Crear perfil del América (última temporada)
america_profile = create_team_profile(df_all_players, 'América', '2024/2025')

print(f"\n{'=' * 60}")
print("PERFIL DEL CLUB AMÉRICA (2024/2025)")
print('=' * 60)
print("\nMétricas promedio por posición (valores normalizados):")
print(america_profile)


PERFIL DEL CLUB AMÉRICA (2024/2025)

Métricas promedio por posición (valores normalizados):
                   player_season_goals_90_norm  player_season_assists_90_norm  \
position_category                                                               
DEF                                  -0.026982                      -0.700736   
FWD                                   0.597014                       1.328475   
GK                                    0.000000                      -0.324541   
MED                                   0.089045                       0.113072   

                   player_season_np_xg_90_norm  \
position_category                                
DEF                                   0.019182   
FWD                                   0.494308   
GK                                   -0.196053   
MED                                   0.155911   

                   player_season_np_shots_90_norm  \
position_category                                   
DEF          

In [9]:
from pathlib import Path
import pyarrow

output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Guardar dataset completo
df_all_players.to_parquet(output_dir / 'all_players_processed.parquet', index=False)
print(f"\nDataset completo guardado en: {output_dir / 'all_players_processed.parquet'}")

# Guardar solo jugadores del América
america_players.to_parquet(output_dir / 'america_players.parquet', index=False)
print(f"Jugadores del América guardados en: {output_dir / 'america_players.parquet'}")

# Guardar perfil del América
america_profile.to_parquet(output_dir / 'america_profile.parquet')
print(f"Perfil del América guardado en: {output_dir / 'america_profile.parquet'}")

# También guardar en CSV para fácil inspección
df_all_players.to_csv(output_dir / 'all_players_processed.csv', index=False)
print(f"CSV guardado en: {output_dir / 'all_players_processed.csv'}")


Dataset completo guardado en: ../data/processed/all_players_processed.parquet
Jugadores del América guardados en: ../data/processed/america_players.parquet
Perfil del América guardado en: ../data/processed/america_profile.parquet
CSV guardado en: ../data/processed/all_players_processed.csv
